
#            Calcul numérique du MGI d'un RRR par PNL avec contraintes #



|![alternative text](robot-rrr3-sol.png)|
|:--:| 
| *  Robot RRR  * |

## Objectif
On se propose de calculer de manière numérique un modèle géométrique inverse (MGI) d'un robot RRR en utilisant les fonctions de $scipy$ $optimize$ et en considérant des contraintes.

La configuration du robot est défini par le vecteur $q = (q_1, q_2, q_3)^t$.

La situation de l'outil est défini par le vecteur $X = (x, y, \theta)^t$ .


## Calcul du MGD
La fonction suivante calcule le modèle géométrique direct:


In [1]:
#################################################   
# Calcul du MGD du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: Xc = vecteur de situation = (x,y,z) 
def mgd(qdeg):
#### Paramètres du robot
    a,b,c=1,1,1
    qrad=np.deg2rad(qdeg)
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])
    x= a*c1*c2 + b*c1*c23
    y= a*s1*c2 + b*s1*c23
    z= a*s2 + b*s23 +c
    Xd=np.asarray([x,y,z])
    return Xd

In [2]:
#################################################   
# Calcul de J(q) du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: jacobienne(q) analytique:
def jacobienne(qdeg):
#### Paramètres du robot
    a,b=1,1
    qrad=np.deg2rad(qdeg)
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])
 
    Ja=np.array([[-a*s1*c2 -b*s1*c23, -a*c1*s2 -b*c1*s23,  -b*c1*s23], 
                [a*c1*c2 + b*c1*c23, -a*s1*s2 -b*s1*s23,  -b*s1*s23], 
                 [0, a*c2 + b*c23, b*c23]])

    return Ja

In [5]:
#################################################   
%matplotlib inline
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d  # Fonction pour la 3D
from mpl_toolkits.mplot3d import proj3d
import numpy as np
import scipy
import time
from scipy.optimize import minimize
from scipy.optimize import root
import scipy.optimize as spop
from scipy import linalg


## 1 - Calcul du MGI sans contraintes

Trouver et utiliser les fonctions de $optimize$ permettant de calculer le MGI $(root, broyden, minimize,...)$.

Comparer les résultats avec votre implémentation


In [15]:
###################################################################################

qinit= np.asarray([5., 0,  4.])

qbut= np.asarray([-45., 0,  45.])
## Calcul Xbut à partir de qbut
#Xbut= np.asarray(mgd(np.radians(qbut)))
Xbut= mgd(qbut)
print("Xbut=", Xbut[0], "Ybut = ", Xbut[1], "Z but = ", Xbut[2])

sol = root(mgd,qinit,jac = jacobienne,method = 'hybr')
sol.x

Xbut= 1.2071067811865477 Ybut =  -1.2071067811865477 Z but =  1.7071067811865475


array([  5., -30., 240.])

## 2 - Calcul du MGI avec  contraintes

Définir et utiliser une fonction $fun$ permettant de calculer la solution du MGI la plus proche de la configuration initiale $qinit$ donnée à la fonction $minimize$.

$res = minimize(fun, qinit, ....)$


## 3 - Calcul du MGI avec  contraintes

Définir et utiliser une fonction permettant de calculer la solution du MGI en prenant en compte les butées du robot.

$q_1 \in [-120, 120]$

$q_2 \in [-45, 120]$

$q_3 \in [-120, 120]$
